In [1]:
import numpy as np
from pydgrid.pydgrid import grid
import ipywidgets as widgets
from lxml import etree as et

## Compensación de reactiva en BT

In [9]:
data = {
        "buses":[
                 {"bus": "B1",  "pos_x":   0, "pos_y":  0, "units": "m", "U_kV":20.0},
                 {"bus": "B2",  "pos_x":  10, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "B3",  "pos_x": 100, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "B4",  "pos_x": 110, "pos_y":  0, "units": "m", "U_kV":0.4},
                 {"bus": "BC",  "pos_x": 100, "pos_y":-10, "units": "m", "U_kV":0.4},
                ],
        "grid_formers":[
                        {"bus": "B1",
                        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                        "kV": [11.547, 11.547, 11.547]}
                       ],
        "transformers":[
                        {"bus_j": "B1",  "bus_k": "B2",  "S_n_kVA": 1000.0, "U_j_kV":20, "U_k_kV":0.4,
                         "R_cc_pu": 0.01, "X_cc_pu":0.04, "connection": "Dyn11",   "conductors_j": 3, "conductors_k": 4},
                       ],
        "lines":[
                 {"bus_j": "B2",  "bus_k": "B3",  "code": "lv_cu_150", "m": 100.0},
                 {"bus_j": "B3",  "bus_k": "B4",  "code": "lv_cu_150", "m":  10.0},
                 {"bus_j": "BC",  "bus_k": "B3",  "code": "lv_cu_150", "m":  10.0},
                ],
        "loads":[{"bus": "B4" , "kVA": 100.0, "pf": 0.85,"type":"3P+N"}
                ],
        "shunts":[
                 {"bus": "B2" , "R": 0.001, "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "BC" , "R": 0.0, "X": -0.0032, "bus_nodes": [1,2]},
                 {"bus": "BC" , "R": 0.0, "X": -0.0032, "bus_nodes": [2,3]},
                 {"bus": "BC" , "R": 0.0, "X": -0.0032, "bus_nodes": [3,1]},
                 ],
        "line_codes":
            {"lv_cu_150":  {"Rph":0.167,"Xph":0.08, "Rn":0.167, "Xn": 0.08}
            }
       }

sld = widgets.FloatSlider(
    value=7.5,
    min=-150,
    max= 150,
    step=1,
    description='Q cap. (kvar):',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

txt_pper =widgets.FloatText(value=7.5,description=r'$ P_{perd}$ (kW): ')
txt_pred =widgets.FloatText(value=7.5,description=r'\(\sf P_{red}\) (kW): ')
txt_qred =widgets.FloatText(value=7.5,description=r'\(\sf Q_{red}\) (kvar): ')
txt_ilinea =widgets.FloatText(value=7.5,description=r'\(\sf I_{linea}\) (A): ')
txt_ucarga =widgets.FloatText(value=7.5,description=r'\(\sf U_{c}\) (V): ')

svg_xml = et.parse('system.svg')
root = svg_xml.getroot()
text_list = root.findall('.//{http://www.w3.org/2000/svg}text')
id2text = {}
for item in text_list:
    tspan = item.findall('.//{http://www.w3.org/2000/svg}tspan')
    id2text.update({item.get('id'):tspan[0]})
    
svg = widgets.HTMLMath(
    value=et.tostring(svg_xml),
)
    
def update(change):
    
    Q_r = change['new']*1000
    X_c = 1000.0
    if abs(Q_r)>0.1:
        X_c = 400**2/Q_r*3

    data['shunts'][1]['X'] = X_c 
    data['shunts'][2]['X'] = X_c        
    data['shunts'][3]['X'] = X_c        
    grid_1 = grid()
    grid_1.read(data)  # Load data
    grid_1.pf()  # solve power flow
    mon_1 = grid_1.monitor('B2','B3')
    mon_2 = grid_1.monitor('B4','B3')
    mon_c = grid_1.monitor('BC','B3')
    perdidas = mon_1.P/1e3-85
    txt_pper.value = f'{perdidas:0.2f}' 
    txt_pred.value = f'{mon_1.P/1e3:0.2f}'
    txt_qred.value = f'{mon_1.Q/1e3:0.2f}'
    txt_ilinea.value = f'{mon_1.I:0.0f}'
    txt_ucarga.value = f'{mon_2.U:0.0f}'   
    id2text['Q_red'].text = f'{mon_1.Q/1e3:3.0f} kvar'
    id2text['P_perd'].text = f'{perdidas:0.2f} kW'
    id2text['I_L'].text = f'{mon_1.I:0.0f} A'
    id2text['I_cap'].text = f'{mon_c.I:3.0f} A'
    id2text['Q_cap'].text = f'{mon_c.Q/1e3:4.0f} kvar'
    id2text['U_carga'].text = f'{mon_2.U:0.0f} V'
    svg.value = et.tostring(svg_xml)

sld.observe(update, names='value')


dashboard = widgets.VBox([svg,sld])
dashboard